In [14]:
from rateLimiter import RateLimiter, set_log_level
import cloudscraper
from tqdm import tqdm
import logging
import timeit
import copy

In [10]:
set_log_level(logging.DEBUG)

2025-03-15 14:13:17,020 - rateLimiter - INFO - Log level set to: 10


In [11]:
items = [
    {"name": "Wrapped Greathelm", "refName": "Wrapped Greathelm", "namespace": "ITEM", "icon": "%NOT_FOUND%", "craftable": {"category": "Helmet"}, "armour": {"ar": [79, 79]}, "w": 2, "h": 2},
{"name": "Wrapped Quarterstaff", "refName": "Wrapped Quarterstaff", "namespace": "ITEM", "icon": "%NOT_FOUND%", "craftable": {"category": "Warstaff"}, "w": 2, "h": 4},
{"name": "Wrapped Sandals", "refName": "Wrapped Sandals", "namespace": "ITEM", "icon": "%NOT_FOUND%", "craftable": {"category": "Boots"}, "armour": {"es": [22, 22]}, "w": 2, "h": 2},
{"name": "Astramentis", "refName": "Astramentis", "namespace": "UNIQUE", "unique": {"base": "Stellar Amulet"}, "icon": "https://web.poecdn.com/gen/image/WzI1LDE0LHsiZiI6IjJESXRlbXMvQW11bGV0cy9BbXVsZXQzNiIsInciOjEsImgiOjEsInNjYWxlIjoxfV0/95da0d04a4/Amulet36.png"},
{"name": "Xoph's Breachstone", "refName": "Xoph's Breachstone", "namespace": "ITEM", "icon": "https://web.poecdn.com/gen/image/WzI1LDE0LHsiZiI6IjJESXRlbXMvQ3VycmVuY3kvQnJlYWNoL0JyZWFjaEZyYWdtZW50c0ZpcmUiLCJzY2FsZSI6MX1d/2fc9d593d6/BreachFragmentsFire.png", "craftable": {"category": "Breachstone"}, "w": 1, "h": 1},
{"name": "Flash Grenade", "refName": "Flash Grenade", "namespace": "GEM", "icon": "%NOT_FOUND%", "craftable": {"category": "Active Skill Gem"}, "w": 1, "h": 1, "gem": {"awakened": False, "transfigured": False}},
]

In [12]:
UNIQUE_FILTER = {"type_filters": {"filters": {"rarity": {"option": "unique"}}}}
NONUNIQUE_FILTER = {"type_filters": {"filters": {"rarity": {"option": "nonunique"}}}}
BASE_PAYLOAD = {
    "query": {
        "status": {"option": "any"},
        "stats": [{"type": "and", "filters": []}],
    },
    "sort": {"price": "asc"},
}

In [13]:
url = "http://www.pathofexile.com/api/trade2/search/Standard"

payload = {
    "query": {
        "status": { "option": "online" },
        "type": "Heavy Belt",
        "stats": [
            {
                "type": "and",
                "filters": []
            }
        ]
    },
    "sort": { "price": "asc" }
}
headers = {"content-type": "application/json"}
session = RateLimiter(debug=True)

In [5]:
session.post(
    url,
    payload=payload,
    headers=headers
)

2025-03-15 11:36:21,441 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): www.pathofexile.com:80
2025-03-15 11:36:21,636 - urllib3.connectionpool - DEBUG - http://www.pathofexile.com:80 "POST /api/trade2/search/Standard HTTP/11" 200 None
2025-03-15 11:36:21,637 - rateLimiter - DEBUG - Waiting for 28.0 | RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=1:window=300:penalty=0>


< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8
< Accept-Language: en-US,en;q=0.5
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 134
< 
< {"query": {"status": {"option": "online"}, "type": "Heavy Belt", "stats": [{"type": "and", "filters": []}]}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Sat, 15 Mar 2025 16:36:38 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
> X-Rate-Limit-Ip: 5:10:60,15:60:300,30:300:1800
> X-Rate-Limit-Ip-State: 1:10:0,1:60:0,1:300:0
> X-Frame-Options: SAMEORIGIN
> Strict-Transport-Security: max-age=63115

2025-03-15 11:36:49,638 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,1:60:0,1:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=1:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=1:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 15, 11, 36, 21, 637560), 'prev_state': None, 'prev_rate_limits': None}})


<Response [200]>

In [17]:
for i in tqdm(items):
    pay = copy.deepcopy(BASE_PAYLOAD)
    if i.get("namespace") == "UNIQUE":
        pay["query"]["filters"] = UNIQUE_FILTER
        pay["query"]["type"] = i.get("unique").get("base")
        pay["query"]["name"] = i.get("refName")
    else:
        pay["query"]["filters"] = NONUNIQUE_FILTER
        pay["query"]["type"] = i.get("refName")
    res = session.post(
        url,
        payload=pay,
        headers=headers
    )

  0%|          | 0/6 [00:00<?, ?it/s]2025-03-15 14:23:40,369 - urllib3.connectionpool - DEBUG - http://www.pathofexile.com:80 "POST /api/trade2/search/Standard HTTP/11" 200 None
2025-03-15 14:23:40,370 - rateLimiter - DEBUG - Waiting for 28.0 | RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=2:window=300:penalty=0>


< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Linux; Android 6.0.1; SM-G532F Build/MMB29T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.126 Mobile Safari/537.36
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
< Accept-Language: en-US,en;q=0.9
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 217
< 
< {"query": {"status": {"option": "any"}, "stats": [{"type": "and", "filters": []}], "filters": {"type_filters": {"filters": {"rarity": {"option": "nonunique"}}}}, "type": "Wrapped Greathelm"}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Sat, 15 Mar 2025 19:23:57 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
> 

2025-03-15 14:24:08,372 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,1:60:0,2:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=2:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=1:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 15, 14, 23, 40, 370741), 'prev_state': '1:10:0,2:60:0,7:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=7:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
 17%|█▋        | 1/6 [00:28<02:21, 28.22s/it]2025-03-15 14:24:08,557 - urllib3.connectionpo

< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Linux; Android 6.0.1; SM-G532F Build/MMB29T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.126 Mobile Safari/537.36
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
< Accept-Language: en-US,en;q=0.9
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 220
< 
< {"query": {"status": {"option": "any"}, "stats": [{"type": "and", "filters": []}], "filters": {"type_filters": {"filters": {"rarity": {"option": "nonunique"}}}}, "type": "Wrapped Quarterstaff"}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Sat, 15 Mar 2025 19:24:25 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip

2025-03-15 14:24:36,559 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,2:60:0,3:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=3:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 15, 14, 24, 8, 558380), 'prev_state': '1:10:0,1:60:0,2:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=2:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=1:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
 33%|███▎      | 2/6 [00:56<01:52, 28.20s/it]2025-03-15 14:24:36,788 - urllib3.connectionpoo

...

2025-03-15 14:25:32,966 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,2:60:0,4:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=4:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 15, 14, 25, 4, 966013), 'prev_state': '1:10:0,2:60:0,4:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=4:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
 67%|██████▋   | 4/6 [01:52<00:56, 28.20s/it]2025-03-15 14:25:33,157 - urllib3.connectionpoo

< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Linux; Android 6.0.1; SM-G532F Build/MMB29T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.126 Mobile Safari/537.36
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
< Accept-Language: en-US,en;q=0.9
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 218
< 
< {"query": {"status": {"option": "any"}, "stats": [{"type": "and", "filters": []}], "filters": {"type_filters": {"filters": {"rarity": {"option": "nonunique"}}}}, "type": "Xoph's Breachstone"}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Sat, 15 Mar 2025 19:25:50 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
>

2025-03-15 14:26:01,159 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,2:60:0,5:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=5:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 15, 14, 25, 33, 157471), 'prev_state': '1:10:0,2:60:0,4:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=4:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
 83%|████████▎ | 5/6 [02:21<00:28, 28.20s/it]2025-03-15 14:26:01,459 - urllib3.connectionpo

< POST /api/trade2/search/Standard HTTP/1.1
< Host: www.pathofexile.com
< User-Agent: Mozilla/5.0 (Linux; Android 6.0.1; SM-G532F Build/MMB29T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.126 Mobile Safari/537.36
< Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8
< Accept-Language: en-US,en;q=0.9
< Accept-Encoding: gzip, deflate
< content-type: application/json
< Content-Length: 213
< 
< {"query": {"status": {"option": "any"}, "stats": [{"type": "and", "filters": []}], "filters": {"type_filters": {"filters": {"rarity": {"option": "nonunique"}}}}, "type": "Flash Grenade"}, "sort": {"price": "asc"}}

> HTTP/1.1 200 OK
> Date: Sat, 15 Mar 2025 19:26:18 GMT
> Content-Type: application/json
> Transfer-Encoding: chunked
> Connection: keep-alive
> Access-Control-Expose-Headers: X-Rate-Limit-Policy, X-Rate-Limit-Rules, X-Rate-Limit-Ip, X-Rate-Limit-Ip-State
> X-Rate-Limit-Policy: trade-search-request-limit
> X-Rate-Limit-Rules: Ip
> X-Ra

2025-03-15 14:26:29,461 - rateLimiter - DEBUG - Current rate limits: defaultdict(None, {'trade-search-request-limit': {'policy': 'trade-search-request-limit', 'rules': 'Ip', 'limit': '5:10:60,15:60:300,30:300:1800', 'state': '1:10:0,2:60:0,6:300:0', 'rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=6:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)], 'last_update': datetime.datetime(2025, 3, 15, 14, 26, 1, 460031), 'prev_state': '1:10:0,2:60:0,5:300:0', 'prev_rate_limits': [(RateLimiter<max=30:window=300:penalty=1800>, RateLimiter<max=5:window=300:penalty=0>), (RateLimiter<max=15:window=60:penalty=300>, RateLimiter<max=2:window=60:penalty=0>), (RateLimiter<max=5:window=10:penalty=60>, RateLimiter<max=1:window=10:penalty=0>)]}})
100%|██████████| 6/6 [02:49<00:00, 28.22s/it]


In [23]:
res = session.session.post(
        url,
        json=payload,
        headers=headers
    )
print(res.status_code)
print(res.text)
print(res.headers)


2025-03-13 21:06:23,872 - urllib3.connectionpool - DEBUG - Resetting dropped connection: www.pathofexile.com
2025-03-13 21:06:24,063 - urllib3.connectionpool - DEBUG - http://www.pathofexile.com:80 "POST /api/trade2/search/Standard HTTP/11" 200 None


200
{"id":"aDaM4V3ue","complexity":5,"result":["fa0665087835d53ea92d73e5684fb19056b31f6a9e6dce6c74b44de289946bd5","a81b2e3ff26541be20bf2affa42077bc82c7c75426efebcf4a84d2b7bf1dd43b","943313b1a5345a82e4bbe0e4dbe3ca90e1d140c87a682d632ef5b2d74a031114","76845362ff73808298f7a1aa53a318d1fcbb8de93d386d5733b4876399e8ca47","fe013e1f4edabe371fc2c71162dfb6c850fcdcad27f334d1cd5a4fdc17d1500a","e610aa90e9b075ff59c14dc3a89a569375d0dc701254e66e42f85b089526e105","0b642c7894480411cd0f976f7898c0706789c01ebbf12dc18b3b600a9e523f9b","4dd58b9011548c04979c6e4ae7a13c5d453d7159acc82a0aded315d5f3f181f6","cc67b5a8d36b47369527666c89a140f8c9b19eaeb535480bcf55fb3a9633d5e0","96915e45f2f015366879bed35ee656ff83336afcfe8d591cb6cf00d38814c495","7476153e864807b4ad4402c61b6719e5d955062ff2393c7623f77bc5e26a8f07","d708a0520ad799ed057578483220b72f705d09dd5a083b626d47f2a903f0ec32","1f90566df8f2cc8314b182286b4566da959e496441ffcf707104aeaf52887f98","17ada987bcca7748b1f364e96dab3b5f9d3a8403032bc9b77379870ecd7453a5","7be5473d401b4c